In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-10-30 22:20:42.034 | INFO     | SiliconDriver   - Detected 4 PCI devices
2023-10-30 22:20:42.094 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 3 device_id: 0xfaca revision: 0)
2023-10-30 22:20:42.100 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 2 device_id: 0xfaca revision: 0)
2023-10-30 22:20:42.104 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 1 device_id: 0xfaca revision: 0)
2023-10-30 22:20:42.112 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-10-30 22:20:42.200 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-30 22:20:42.200 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-30 22:20:42.200 | INFO  

In [2]:
m = 1024
k = 1024
n = 1024

# Initialize tensors a and b with random values using torch

In [3]:
torch_a = torch.randn((m, k), dtype=torch.bfloat16)
torch_b = torch.randn((k, n), dtype=torch.bfloat16)

In [4]:
a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device)
b = ttnn.to_device(b, device)

# Matrix multiply tensor a and b
The operation will run longer the first time because the kernels need to get compiled

In [ ]:
%%timeit -n 1 -r 1
output = a @ b

In [ ]:
%%timeit -n 2 -r 5
output = a @ b

# Inspect the result of the matrix multiply

In [ ]:
output = a @ b
print("Printing ttnn tensor")
print(output.shape)
print(output[:1, :32])

# Tilize tensors before running matrix multiply

In [ ]:
a = ttnn.to_layout(a, ttnn.TILE_LAYOUT)
b = ttnn.to_layout(b, ttnn.TILE_LAYOUT)

The operation will execute much faster because it will re-use compiled kernels

In [ ]:
%%timeit -n 2 -r 5
output = a @ b

# Matrix multiply tensor a and b by using more performant config

In [ ]:
a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device, memory_config=ttnn.L1_MEMORY_CONFIG)
b = ttnn.to_device(b, device, memory_config=ttnn.L1_MEMORY_CONFIG)

a = ttnn.to_layout(a, ttnn.TILE_LAYOUT)
b = ttnn.to_layout(b, ttnn.TILE_LAYOUT)

In [ ]:
%%timeit -n 1 -r 1
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=(8, 8))

In [ ]:
%%timeit -n 2 -r 5
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=(8, 8))

# Close the device

In [ ]:
ttnn.close(device)